In [64]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"

In [65]:
import torch
from sklearn.datasets import make_moons

In [66]:
def problem_moons(train=True):
    num_samples = 10000 if train else 1000 * 100
    num_dimensions = 50
    x_, y_ = make_moons(n_samples=num_samples, noise=0.07)
    x_[:, [0, 1]] = x_[:, [1, 0]]
    x_[:, 0] *= -1
    x_ *= 1.1
    x_ = x_ - x_.mean(0, keepdims=True)
    x_[:, 0] *= 1.2
    x = torch.FloatTensor(x_)
    y = torch.FloatTensor(y_).view(-1, 1)
    is_maj = torch.zeros(num_samples)
    is_maj = torch.logical_or(((2 * y - 1) * torch.sign(x[:, 0:1])) > 0, (torch.abs(x[:, 1:2]) > 1))
    if not train:
        i_min = torch.where(is_maj == 0)[0][:900]
        i_maj = torch.where(is_maj == 1)[0][:100]
        i_all = torch.cat((i_min, i_maj))[torch.randperm(1000)]
        x, y, is_maj = x[i_all], y[i_all], is_maj[i_all]
    noise = torch.randn(len(x), num_dimensions - 2)
    x = torch.cat((x, noise), 1)
    return x, y.view(-1).long(), is_maj.view(-1)


def build_network(x_tr, y_tr):
    return torch.nn.Sequential(
        torch.nn.Linear(x_tr.size(1), 200), torch.nn.ReLU(), torch.nn.Linear(200, 200),
        torch.nn.ReLU(), torch.nn.Linear(200, y_tr.size(1))
    )


def train_network(network, x_tr, y_tr, n_iterations=1000, lr=1e-1):
    optimizer = torch.optim.SGD(network.parameters(), lr=lr)
    loss = torch.nn.BCEWithLogitsLoss()
    for iteration in range(n_iterations):
        optimizer.zero_grad()
        loss(network(x_tr), y_tr).backward()
        optimizer.step()

def accuracy(network, x, y):
    return network(x).gt(0).eq(y).float().mean().item()

def accuracy_ens(networks, x, y):
    for i, network in enumerate(networks):
        print("net" + str(i), accuracy(network, x, y))
        
    list_preds = [network(x) for network in networks]
    preds = torch.mean(torch.stack(list_preds, dim=0), 0)
    print("ense", preds.gt(0).eq(y).float().mean().item())

In [67]:
problem = problem_moons
x_tr, y_tr, m_tr = problem(train=True)
y_tr = y_tr.view(-1, 1).float()
x_te, y_te, m_te = problem(train=False)
y_te = y_te.view(-1, 1).float()


In [ ]:
# todo balancing

In [ ]:
x, y = x_tr[27:27+1], y_tr[27:27+1]

In [ ]:
torch.logical_or(((2 * y - 1) * torch.sign(x[:, 0:1])) > 0, (torch.abs(x[:, 1:2]) > 1))

In [ ]:
(torch.abs(x[:, 1:2]) > 1)

In [ ]:
print(x,y)

In [ ]:
x_tr[25], y_tr[25]

In [ ]:
torch.where(~m_tr)

In [ ]:
START_0=0
END_0=800

networks = []
for num_network in range(20):
    net = build_network(x_tr, y_tr)
    train_network(net, x_tr[START_0:END_0], y_tr[START_0:END_0], lr=1e-1)
    networks.append(net)

In [ ]:
START_1=100
END_1=900

networks_1 = []
for num_network in range(20):
    net = build_network(x_tr, y_tr)
    train_network(net, x_tr[START_1:END_1], y_tr[START_1:END_1], lr=1e-1)
    networks_1.append(net)

In [ ]:
networks_01 = networks[:10] + networks_1[:10]

In [ ]:
print("train")
accuracy_ens(networks, x_tr[END_1:], y_tr[END_1:])
print("\ntest")
accuracy_ens(networks, x_te, y_te)

In [ ]:
print("train")
accuracy_ens(networks_01, x_tr[END_1:], y_tr[END_1:])
print("\ntest")
accuracy_ens(networks_01, x_te, y_te)

In [ ]:
print("train")
accuracy_ens(networks_1, x_tr[END_1:], y_tr[END_1:])
print("\ntest")
accuracy_ens(networks_1, x_te, y_te)